In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = "Alveolar_cells_Type_II_Merged_Batches"
df = pd.read_csv("{}/mathematica/mathematica_results.csv".format(dataset))

In [ ]:
df.thresholds[df.thresholds.str.startswith("\"\"")] = np.nan
df.loc[df.thresholds == "NotFound", "thresholds"] = np.nan

In [ ]:
df.thresholds = df.thresholds.astype("float")

In [ ]:
df

In [ ]:
df[~df.antimode.str.contains("Mixture")]

In [ ]:
print("Total genes considered:",df.shape[0])
print("SystemException:",df[df.distribution == "-1"].shape[0])
print("Not Mixtures:",df[df.antimode.str.contains("Mixture")].shape[0])
print("Contains Laplace:",df[df.antimode.str.contains("Lapla")].shape[0])
print("Contains Zipf:",df[df.antimode.str.contains("Zipf")].shape[0])
print("Contains Uniform:",df[df.antimode.str.contains("Uniform")].shape[0])
print("Contains Benford:",df[df.antimode.str.contains("Benford")].shape[0])
print("Contains LogSeries:",df[df.antimode.str.contains("LogSeries")].shape[0])
print("Hand Removed:",df[df.antimode.str.contains("HandRemoved")].shape[0])
print("Antimode Found:",df.thresholds.count())
print("Antimode Found < 0.5:",(df.thresholds < 0.5).sum())

In [ ]:
df.distribution.str.split("[").apply(lambda x: x[0]).value_counts()

In [ ]:
df.rename({"Unnamed: 0": "gene_id"}, axis=1, inplace=True)

In [ ]:
df[df.gene_id.str.startswith("Pcdh")]

In [ ]:
_df = df.dropna().drop(["distribution", "antimode"],axis=1).copy()
_df["uncorrected_threshold"] = _df.thresholds.copy()
_df["threshold"] = _df.thresholds.copy()
_df.loc[_df.threshold < 0.5,"threshold"] = 0.5
_df.drop("thresholds",inplace=True,axis=1)

In [ ]:
_df.to_csv("{}/mathematica/fitted_distributions_thresholds.csv".format(dataset),index=False)

In [ ]:
all_genes_df = pd.read_csv("{}.csv".format(dataset),index_col="gene_id").loc[_df.gene_id]

In [ ]:
for _, (gene_id, _, threshold) in _df.iterrows():
    all_genes_df.loc[gene_id,all_genes_df.loc[gene_id,:] < threshold] = 0
    all_genes_df.loc[gene_id,all_genes_df.loc[gene_id,:] >= threshold] = 1

In [ ]:
all_genes_df.to_csv("{}/mathematica/fitted_distributions_dichotomised.csv".format(dataset))

In [ ]:
_df.uncorrected_threshold.describe()

In [ ]:
_df